ステップ0: 初期設定とアクセストークンの取得



In [1]:
# -*- coding: utf-8 -*-
import requests
import json
import time
import pandas as pd
from google.colab import userdata
from datetime import datetime, timedelta
import urllib.parse

def get_access_token():
    """SP-APIのアクセストークンを取得します。"""
    try:
        refresh_token = userdata.get("SP_API_REFRESH_TOKEN")
        client_id = userdata.get("SP_API_CLIENT_ID")
        client_secret = userdata.get("SP_API_CLIENT_SECRET")
        if not all([refresh_token, client_id, client_secret]):
            print("エラー: ColabのSecretsに必要な情報が設定されていません。")
            return None
        token_response = requests.post(
            url="https://api.amazon.com/auth/o2/token",
            headers={"Content-Type": "application/x-www-form-urlencoded"},
            data={"grant_type": "refresh_token", "refresh_token": refresh_token, "client_id": client_id, "client_secret": client_secret,},
        )
        token_response.raise_for_status()
        return token_response.json().get("access_token")
    except requests.exceptions.RequestException as e:
        print(f"アクセストークンの取得に失敗しました: {e}")
        if e.response is not None: print(f"レスポンス: {e.response.text}")
        return None
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")
        return None

# --- メイン処理の開始 ---
access_token = get_access_token()
if access_token:
    print("アクセストークンの取得に成功しました。")
    #@markdown ---
    #@markdown ### 注文レポートのパラメータ設定
    #@markdown `YYYY-MM-DD`の形式で取得したい期間の開始日と終了日を入力してください。
    CreatedAfter_str = '2025-12-01' #@param {type:"date"}
    CreatedBefore_str = '2025-12-07' #@param {type:"date"}
    MarketplaceIds = 'A1VC38T7YXB528' #@param {type:"string"}

    headers = {'Content-Type': 'application/json', 'x-amz-access-token': access_token}
    print("変数の設定が完了しました。次のステップに進んでください。")
else:
    print("アクセストークンが取得できなかったため、処理を中断します。")

アクセストークンの取得に成功しました。
変数の設定が完了しました。次のステップに進んでください。


ステップ1: 注文データの取得とスプレッドシートへの保存

In [2]:
# -*- coding: utf-8 -*-
# 必要なライブラリを静かにインストール
!pip install gspread gspread-dataframe -q

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
from urllib.parse import urlencode
import numpy as np # NaNをNoneに置換するために必要

if 'access_token' in locals() and access_token:
    try:
        # --- スプレッドシートの準備（毎回クリア） ---
        print("\n--- Googleスプレッドシートの準備を開始します ---")
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        print("認証に成功しました。")

        spreadsheet_name = 'SP-API Orders Report'
        orders_worksheet_name = 'orders'

        try:
            spreadsheet = gc.open(spreadsheet_name)
            print(f"既存のスプレッドシート '{spreadsheet_name}' を開きました。")
        except gspread.exceptions.SpreadsheetNotFound:
            spreadsheet = gc.create(spreadsheet_name)
            print(f"スプレッドシート '{spreadsheet_name}' を新規作成しました。")

        # ワークシートを取得または作成し、毎回クリアする
        def get_or_create_worksheet(name):
            try:
                sheet = spreadsheet.worksheet(name)
            except gspread.exceptions.WorksheetNotFound:
                sheet = spreadsheet.add_worksheet(title=name, rows=1, cols=1)
            sheet.clear() # 毎回シートの内容をクリア
            return sheet

        orders_worksheet = get_or_create_worksheet(orders_worksheet_name)
        print(f"'{orders_worksheet_name}' シートをクリアし、準備が完了しました。")

        # --- 全注文データを取得 (ページネーション対応) ---
        print("\n--- 注文データの取得を開始します ---")
        all_orders = []
        base_url = "https://sellingpartnerapi-fe.amazon.com/orders/v0/orders"
        created_after = datetime.fromisoformat(CreatedAfter_str).isoformat()
        created_before = (datetime.fromisoformat(CreatedBefore_str) + timedelta(days=1) - timedelta(seconds=1)).isoformat()
        params = {'MarketplaceIds': MarketplaceIds, 'CreatedAfter': created_after, 'CreatedBefore': created_before}

        page_num = 1
        while True:
            query_string = urlencode(params, safe=':')
            request_url = f"{base_url}?{query_string}"
            print(f"ページ {page_num}: 注文リストを取得中...")
            response = requests.get(request_url, headers=headers)
            response.raise_for_status()
            data = response.json()
            all_orders.extend(data['payload'].get('Orders', []))

            if 'NextToken' in data['payload']:
                params = {'NextToken': data['payload']['NextToken']}
                page_num += 1
                time.sleep(1.5) # APIレート制限を考慮
            else:
                break
        print(f"  -> 期間内の注文を {len(all_orders)} 件取得しました。")

        # --- スプレッドシートに一括で書き込み ---
        print("\n--- データの整形とスプレッドシートへの書き込みを開始します ---")

        if all_orders:
            # 取得したデータをDataFrameに変換し、エラーの原因となるNaNをNone(空欄)に置換
            orders_df = pd.json_normalize(all_orders).replace({np.nan: None})

            print(f"  -> '{orders_worksheet_name}'シートに {len(orders_df)} 行のデータを書き込み中...")
            set_with_dataframe(orders_worksheet, orders_df, include_index=False, resize=True)
            print("  -> 書き込みが完了しました。")
        else:
            print("  -> 期間内に注文データが見つからなかったため、書き込みは行いませんでした。")

        print(f"\nすべての処理が完了しました。")
        print(f"下記URLからスプレッドシートを確認してください:\n{spreadsheet.url}")

    except Exception as e:
        print(f"\n処理中に予期せぬエラーが発生しました: {e}")
else:
    print("エラー: ステップ0を先に実行してアクセストークンを取得してください。")


--- Googleスプレッドシートの準備を開始します ---
認証に成功しました。
既存のスプレッドシート 'SP-API Orders Report' を開きました。
'orders' シートをクリアし、準備が完了しました。

--- 注文データの取得を開始します ---
ページ 1: 注文リストを取得中...
  -> 期間内の注文を 46 件取得しました。

--- データの整形とスプレッドシートへの書き込みを開始します ---
  -> 'orders'シートに 46 行のデータを書き込み中...
  -> 書き込みが完了しました。

すべての処理が完了しました。
下記URLからスプレッドシートを確認してください:
https://docs.google.com/spreadsheets/d/1tOCnB22RBcv0h_Cgt6TTHJC9PlbsMYeF5yzTYGbKTiA
